In [1]:
from tqdm import tqdm
import os
from bs4 import BeautifulSoup
from lxml import etree
from pdb import set_trace
import pickle
from multiprocessing import Pool
from itertools import repeat
import itertools
import numpy as np
import pandas as pd


In [2]:
# ignore this
# This is test

html_list = os.listdir('data/raw')
html = 'investing_btc_1365.html'
soup = BeautifulSoup(open('data/raw/{}'.format(html), encoding="ISO-8859-1"), 'html.parser')
dom = etree.HTML(str(soup))
msg_body_list = dom.xpath('//div[@class="comment js-comment"]')
tmp_dict = {}
    
for msg_body in msg_body_list:
    
        msg_text = msg_body.xpath('.//div[@class="mainComment js-content"]/div//span[@class="js-text"]')
        msg_text = msg_text[0].xpath('./text() | ./a/text()')

        main_msg_id = msg_body.xpath('./@data-comment-id')
        main_msg_id = main_msg_id[0].split('-')[-1]

        main_poster = msg_body.xpath('.//div[@class="mainComment js-content"]//span[@class="commentUsername"]/a/text()')
        if len(main_poster) == 0: continue;

        main_poster_url = msg_body.xpath('.//div[@class="mainComment js-content"]//span[@class="commentUsername"]/a/@href')
        main_poster_id = main_poster_url[0].split('/')[-1]

        main_timestamp = msg_body.xpath('.//div[@class="mainComment js-content"]//span[@class="js-date"]/@comment-date-formatted')[0]

        tmp_dict[main_msg_id] = {'msg_id': main_msg_id, 'poster': main_poster, 'poster_url': main_poster_url,
                                 'poster_id': main_poster_id, 'timestamp': main_timestamp, 'msg': msg_text[0]}

        # reply 
        reply_texts = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="js-text"]')
        reply_texts = [reply_text.xpath('./text() | ./a/text()') for reply_text in reply_texts]
        if len(reply_texts) == 0: continue;

        reply_msg_ids = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]')
        reply_msg_ids = [reply_msg_id.xpath('.//div[@class="js-text-wrapper commentText"]/@id | .//div[@class="js-text-wrapper commentText withImage"]/@id')[0] for reply_msg_id in reply_msg_ids]

        reply_posters = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="commentUsername"]/a/text()')
        reply_poster_urls = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="commentUsername"]/a/@href') 

        reply_timestamps = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="js-date"]/@comment-date-formatted')

        if (len(reply_texts) != len(reply_msg_ids)): print(reply_texts, reply_msg_ids)

        for i, reply_text in enumerate(reply_texts):
            reply_msg_id = reply_msg_ids[i].split('-')[-1]
            reply_poster_id = reply_poster_urls[i].split('/')[-1]
            reply_poster = reply_posters[i]
            reply_timestamp = reply_timestamps[i]
            reply_poster_url = reply_poster_urls[i]
            tmp_dict[reply_msg_id] = {'msg_id': reply_msg_id, 'poster': reply_poster, 'poster_url': reply_poster_url, 'poster_id': reply_poster_id, 
                                      'reply_id': main_msg_id, 'timestamp': reply_timestamp, 'msg': reply_text}

# print(tmp_dict['6961311'])

In [3]:
def make_post_dict(html): 

    tmp_dict = {}
    
    if 'html' not in html: return;
    
    soup = BeautifulSoup(open('data/raw/{}'.format(html), encoding="ISO-8859-1"), 'html.parser')
    dom = etree.HTML(str(soup))
    msg_body_list = dom.xpath('//div[@class="comment js-comment"]')
    for msg_body in msg_body_list:

        msg_text = msg_body.xpath('.//div[@class="mainComment js-content"]/div//span[@class="js-text"]')
        msg_text = msg_text[0].xpath('./text() | ./a/text()')

        main_msg_id = msg_body.xpath('./@data-comment-id')
        main_msg_id = main_msg_id[0].split('-')[-1]

        main_poster = msg_body.xpath('.//div[@class="mainComment js-content"]//span[@class="commentUsername"]/a/text()')
        if len(main_poster) == 0: continue;

        main_poster_url = msg_body.xpath('.//div[@class="mainComment js-content"]//span[@class="commentUsername"]/a/@href')
        main_poster_id = main_poster_url[0].split('/')[-1]

        main_timestamp = msg_body.xpath('.//div[@class="mainComment js-content"]//span[@class="js-date"]/@comment-date-formatted')[0]

        tmp_dict[main_msg_id] = {'msg_id': main_msg_id, 'poster': main_poster[0], 'poster_url': main_poster_url[0],
                                 'poster_id': main_poster_id, 'timestamp': main_timestamp, 'msg': ' '.join(msg_text)}

        # reply 
        reply_texts = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="js-text"]')
        reply_texts = [reply_text.xpath('./text() | ./a/text()') for reply_text in reply_texts]
        if len(reply_texts) == 0: continue;

        reply_msg_ids = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]')
        reply_msg_ids = [reply_msg_id.xpath('.//div[@class="js-text-wrapper commentText"]/@id | .//div[@class="js-text-wrapper commentText withImage"]/@id')[0] for reply_msg_id in reply_msg_ids]

        reply_posters = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="commentUsername"]/a/text()')
        reply_poster_urls = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="commentUsername"]/a/@href') 

        reply_timestamps = msg_body.xpath('.//div[@class="commentReply js-comment js-comment-reply"]//span[@class="js-date"]/@comment-date-formatted')

        if (len(reply_texts) != len(reply_msg_ids)): print(reply_texts, reply_msg_ids)

        for i, reply_text in enumerate(reply_texts):
            reply_msg_id = reply_msg_ids[i].split('-')[-1]
            reply_poster_id = reply_poster_urls[i].split('/')[-1]
            reply_poster = reply_posters[i]
            reply_timestamp = reply_timestamps[i]
            reply_poster_url = reply_poster_urls[i]
            tmp_dict[reply_msg_id] = {'msg_id': reply_msg_id, 'poster': reply_poster, 'poster_url': reply_poster_url, 'poster_id': reply_poster_id, 
                                      'reply_id': main_msg_id, 'timestamp': reply_timestamp, 'msg': ' '.join(reply_text)}

    return tmp_dict


In [4]:
# post_dict: 
# key:  msg_id
# value: dict{msg_id:'', poster:'', poster_url: '', poster_id:'', reply_id:'',  msg:''}
posts_dict = {}

html_list = os.listdir('data/raw')

pool = Pool(processes=32)

# with tqdm(total=len(html_list)) as pbar:
#     for _ in tqdm(pool.imap_unordered(make_post_dict, html_list)):
#         pbar.update()
dict_list = pool.map(make_post_dict, html_list)
pool.close() 
pool.join()

In [5]:
for tmp_dict in tqdm(dict_list):
    if tmp_dict == None: continue;
    for k, v in tmp_dict.items():
        posts_dict[k] = v

100%|██████████| 6962/6962 [00:00<00:00, 49132.21it/s]


In [6]:
len(posts_dict)

403654

In [7]:
if not os.path.exists('data/parsed/'):
    os.mkdir('data/parsed')
pickle.dump(posts_dict, open("data/parsed/investing_posts_dict", "wb"))

In [8]:
with open("data/parsed/investing_posts_dict", "rb") as f:
    posts_dict = pickle.load(f)


In [9]:
len(posts_dict)

403654

In [10]:
posts_dict['31211866']

{'msg_id': '31211866',
 'poster': 'Uguysarefool',
 'poster_url': '/members/contributors/204832924',
 'poster_id': '204832924',
 'reply_id': '31211821',
 'timestamp': '',
 'msg': 'Marthinus De Beer  \xa0just picking a random comment by u\nMay 16, 2021 4:44PM ET\nCryptoð\x9fª\x99 â\x9a°ð\x9fª¦\nPlease, u are just a ð\x9f¤¡'}

In [11]:
len(posts_dict['31211866']['msg'])

122

In [12]:
df = pd.DataFrame(posts_dict.values())
df

,msg_id,poster,poster_url,poster_id,timestamp,msg,reply_id
0,7329237,Bitcoin Bull,/members/contributors/202158045,202158045,"Dec 10, 2017 5:39PM ET",Fear of missing out FOMO factor is big! Buying...,NaN
1,7329232,Gray Box,/members/contributors/200518803,200518803,"Dec 10, 2017 5:38PM ET",For the longs on the 1hr buy signal at 13667 o...,NaN
2,7329240,Gray Box,/members/contributors/200518803,200518803,"Dec 10, 2017 5:38PM ET",This is a look back sort of thing not trade ex...,7329232
3,7329247,Brandon Becerra,/members/contributors/201744932,201744932,"Dec 10, 2017 5:38PM ET",Thx brother,7329232
4,7329265,Richard Mcglocklin,/members/contributors/201671886,201671886,"Dec 10, 2017 5:38PM ET",Thanks Gray,7329232
...,...,...,...,...,...,...,...
403649,9334723,Jared Foxman,/members/contributors/203227362,203227362,"Jun 29, 2018 6:31PM ET",By bears,NaN
403650,9334719,Crypto Short,/members/contributors/202884766,202884766,"Jun 29, 2018 6:29PM ET",Enjoy the bully weekend and trade safe instead!,NaN
403651,9334685,David Goldberg,/members/contributors/203106630,203106630,"Jun 29, 2018 6:13PM ET",Chart is nonsense. Good luck. https://invst.l...,NaN
403652,9334682,Crypto Short,/members/contributors/202884766,202884766,"Jun 29, 2018 6:12PM ET",BCH is your first pump!,NaN
